In [39]:
#from matplotlib import pyplot as plt
import numpy as np
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.metrics import confusion_matrix, f1_score, precision_score, accuracy_score
from db import psqlServer
sv = psqlServer()

In [16]:
AI = "arx_AI"        # artificial intelligence
FA = "arx_math_FA"   # functional analysis
GR = "arx_GRQC"      # general relativity and quantum cosmology
LG = "arx"           # learning
NT = "arx_math_NT"   # number theory

In [21]:
# How many items in all tables?
UNION = " UNION ALL "
SEL_C = "SELECT COUNT(*) as FOO FROM "
sum_statement = "SELECT SUM(FOO) FROM ("
sum_statement += SEL_C + AI
sum_statement += UNION + SEL_C + FA 
sum_statement += UNION + SEL_C + GR 
sum_statement += UNION + SEL_C + LG 
sum_statement += UNION + SEL_C + NT
sum_statement += ") as B;"

total = sv.execute(sum_statement)[0]['sum']
print(total)

116060


In [17]:
# Per table
SEL_C = "SELECT COUNT(*) FROM "
total_AI = sv.execute(SEL_C + AI)[0]['count']
total_FA = sv.execute(SEL_C + FA)[0]['count']
total_GR = sv.execute(SEL_C + GR)[0]['count']
total_LG = sv.execute(SEL_C + LG)[0]['count']
total_NT = sv.execute(SEL_C + NT)[0]['count']

print("AI:{}".format(total_AI))
print("FA:{}".format(total_FA))
print("GR:{}".format(total_GR))
print("LG:{}".format(total_LG))
print("NT:{}".format(total_NT))

AI:13000
FA:16000
GR:50000
LG:15060
NT:22000


In [18]:
# Get some rows from the AI table
rows = sv.execute("SELECT * FROM arx LIMIT 1000")
corpus = [x['abstract'] for x in rows]
labels = [x['has_journal_ref'] for x in rows]
rows[0].keys()

dict_keys(['pub_date', 'title', 'num_pages', 'num_figs', 'journal_ref', 'category', 'author', 'abstract', 'id', 'num_auth', 'summary_length', 'summary_wc', 'has_journal_ref', 'category_other'])

In [19]:
vectorizer = HashingVectorizer(stop_words='english', alternate_sign=False)

In [20]:
vectorizer

HashingVectorizer(alternate_sign=False, analyzer='word', binary=False,
         decode_error='strict', dtype=<class 'numpy.float64'>,
         encoding='utf-8', input='content', lowercase=True,
         n_features=1048576, ngram_range=(1, 1), non_negative=False,
         norm='l2', preprocessor=None, stop_words='english',
         strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
         tokenizer=None)

In [21]:
X = vectorizer.fit_transform(corpus)

In [22]:
X

<1000x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 59474 stored elements in Compressed Sparse Row format>

In [23]:
#analyze = vectorizer.build_analyzer()

In [24]:
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [25]:
tfidf = transformer.fit_transform(X)


/Users/alex/code/article-classifier/ac/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1067: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(float(n_samples) / df) + 1.0


In [26]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, labels, test_size=.6, random_state=4)

In [27]:
X_train

<400x1048576 sparse matrix of type '<class 'numpy.float64'>'
	with 24267 stored elements in Compressed Sparse Row format>

In [31]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [34]:
y_predict = knn.predict(X_test)
f1_score(y_test,y_predict)

0.40771349862258954

In [35]:
confusion_matrix(y_test, y_predict)

array([[311, 101],
       [114,  74]])

In [36]:
precision_score(y_test, y_predict)

0.42285714285714288

In [53]:
clf = GaussianNB()
clf.fit(X_train.toarray(), y_train)

GaussianNB(priors=None)

In [54]:
y_predict = clf.predict(X_test.toarray())
f1_score(y_test,y_predict)

0.31210191082802546

In [55]:
confusion_matrix(y_test, y_predict)

array([[335,  77],
       [139,  49]])

In [48]:
# Decision tree
model = tree.DecisionTreeClassifier()
model.fit(X_train, y_train)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [50]:
y_predict = model.predict(X_test)
f1_score(y_test,y_predict)

0.35327635327635332

In [51]:
confusion_matrix(y_test, y_predict)

array([[311, 101],
       [126,  62]])